# 1. Install dependencies and setup

In [1]:
!pip install tflearn
!conda install -y gdown


import math
import pandas as pd
import numpy as np
import os
import shutil
import cv2
import keras
from tqdm import tqdm
import csv  
# from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
from random import shuffle
# from google.colab import drive
import tensorflow as tf
import tflearn
from keras.preprocessing.image import ImageDataGenerator
from keras import Sequential
from keras.models import Model
from keras import activations
from keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    MaxPooling2D, GlobalAveragePooling2D,\
    Flatten, Rescaling, BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.models import save_model
from keras.utils import data_utils
from keras.layers import VersionAwareLayers
from tflearn.layers.core import input_data
from tensorflow.keras import layers
from keras.engine import training

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 550.2 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=0fd95fee8bc81b635416469915f7d12cc1a6b55e38e62b8bba04a660f32a8e75
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 22.11.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - gdown


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.12.7  |       ha878542_0         143 KB  conda-forge
    certifi-2022.12.7          |     pyhd8ed1ab_0  


# 2. Loading the data via `gdrive`

In [2]:
# drive.mount('/content/drive')
!gdown --id 1s07aL-7nvhO8ESJy_uTCZJMF5lw5LAGK
!unzip "/kaggle/working/NN Dataset.zip"

train_dir ='/kaggle/working/Train'
test_dir ='/kaggle/working/Test'

train_filenames = os.listdir(train_dir)
test_filenames = os.listdir(test_dir)

labels = sorted(set([fname.split('_')[0] for fname in train_filenames]))

/opt/conda/lib/python3.7/site-packages/gdown/cli.py:125: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1s07aL-7nvhO8ESJy_uTCZJMF5lw5LAGK
To: /kaggle/working/NN Dataset.zip
100%|█████████████████████████████████████████| 274M/274M [00:01<00:00, 171MB/s]
Archive:  /kaggle/working/NN Dataset.zip
  inflating: Test/0.jpg              
  inflating: Test/1.jpg              
  inflating: Test/10.jpg             
  inflating: Test/100.jpg            
  inflating: Test/101.jpg            
  inflating: Test/102.jpg            
  inflating: Test/103.jpg            
  inflating: Test/104.jpg            
  inflating: Test/105.jpg            
  inflating: Test/106.jpg            
  inflating: Test/107.jpg            
  inflating: Test/108.jpg            
  inflating: Test/109.jpg            
  inflating: Test/11.jpg             
 

# 3. Preprocessing

1.   Converting all images to `jpg`
2.   Resizing all images to `(255,255,3)` 
3.   Placing each class in a subfolder
4.   Augmenting dataset




### 3.1 Placing each class in a subfolder 

In [3]:
# Create a sub-folder for each label
labels = sorted(set([fname.split('_')[0] for fname in train_filenames]))

for label in labels:
  os.mkdir(f'{train_dir}/{label}')


# Move each image to it's corresponding sub-folder
for file in train_filenames:
  label = file.split('_')[0]
  shutil.move(f'{train_dir}/{file}', f'{train_dir}/{label}/{file}')

# 4. Preparing train/ validation/ test data to feed the model



Converting train/ val to `BatchDataset`



In [4]:
IMG_SIZE = 250

# Get the data
trainIMG = ImageDataGenerator(validation_split=0.2,
                            height_shift_range=0.2,
                            zoom_range=0.2,
                            rotation_range = 40,
                            vertical_flip=True)
trainingDataset = trainIMG.flow_from_directory(directory="/kaggle/working/Train",target_size=(IMG_SIZE,IMG_SIZE),subset='training')


validationnDataset = trainIMG.flow_from_directory(directory="/kaggle/working/Train",target_size=(IMG_SIZE,IMG_SIZE),subset='validation')


Found 1346 images belonging to 6 classes.
Found 335 images belonging to 6 classes.


# 5. Network Architecture

In [5]:
def conv2d_bn(
    x, filters, num_row, num_col, padding="same", strides=(1, 1), name=None
):
    """Utility function to apply conv + BN.

    Args:
      x: input tensor.
      filters: filters in `Conv2D`.
      num_row: height of the convolution kernel.
      num_col: width of the convolution kernel.
      padding: padding mode in `Conv2D`.
      strides: strides in `Conv2D`.
      name: name of the ops; will become `name + '_conv'`
        for the convolution and `name + '_bn'` for the
        batch norm layer.

    Returns:
      Output tensor after applying `Conv2D` and `BatchNormalization`.
    """
    if name is not None:
        bn_name = name + "_bn"
        conv_name = name + "_conv"
    else:
        bn_name = None
        conv_name = None
    
    bn_axis = 3
    x = layers.Conv2D(
        filters,
        (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name,
    )(x)
    x = layers.BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    x = layers.Activation("relu", name=name)(x)
    return x

In [6]:
layers = VersionAwareLayers()


channel_axis = -1
modelWithweights = Sequential()
img_input = layers.Input(shape=[IMG_SIZE, IMG_SIZE, 3])

modelLayers = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding="valid")
modelLayers = conv2d_bn(modelLayers, 32, 3, 3, padding="valid")
modelLayers = conv2d_bn(modelLayers, 64, 3, 3)
modelLayers = layers.MaxPooling2D((3, 3), strides=(2, 2))(modelLayers)

modelLayers = conv2d_bn(modelLayers, 80, 1, 1, padding="valid")
modelLayers = conv2d_bn(modelLayers, 192, 3, 3, padding="valid")
modelLayers = layers.MaxPooling2D((3, 3), strides=(2, 2))(modelLayers)

# mixed 0: 35 modelLayers 35 modelLayers 256
branch1x1 = conv2d_bn(modelLayers, 64, 1, 1)

branch5x5 = conv2d_bn(modelLayers, 48, 1, 1)
branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

branch3x3dbl = conv2d_bn(modelLayers, 64, 1, 1)
branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

branch_pool = layers.AveragePooling2D((3, 3), strides=(1, 1), padding="same")(modelLayers)
branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
modelLayers = layers.concatenate(
    [branch1x1, branch5x5, branch3x3dbl, branch_pool],
    axis=channel_axis,
    name="mixed0",
)

# mixed 1: 35 modelLayers 35 modelLayers 288
branch1x1 = conv2d_bn(modelLayers, 64, 1, 1)

branch5x5 = conv2d_bn(modelLayers, 48, 1, 1)
branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

branch3x3dbl = conv2d_bn(modelLayers, 64, 1, 1)
branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

branch_pool = layers.AveragePooling2D((3, 3), strides=(1, 1), padding="same")(modelLayers)
branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
modelLayers = layers.concatenate(
    [branch1x1, branch5x5, branch3x3dbl, branch_pool],
    axis=channel_axis,
    name="mixed1",
)

# mixed 2: 35 modelLayers 35 modelLayers 288
branch1x1 = conv2d_bn(modelLayers, 64, 1, 1)

branch5x5 = conv2d_bn(modelLayers, 48, 1, 1)
branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

branch3x3dbl = conv2d_bn(modelLayers, 64, 1, 1)
branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

branch_pool = layers.AveragePooling2D(
    (3, 3), strides=(1, 1), padding="same"
)(modelLayers)
branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
modelLayers = layers.concatenate(
    [branch1x1, branch5x5, branch3x3dbl, branch_pool],
    axis=channel_axis,
    name="mixed2",
)

# mixed 3: 17 modelLayers 17 modelLayers 768
branch3x3 = conv2d_bn(modelLayers, 384, 3, 3, strides=(2, 2), padding="valid")

branch3x3dbl = conv2d_bn(modelLayers, 64, 1, 1)
branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3, strides=(2, 2), padding="valid")

branch_pool = layers.MaxPooling2D((3, 3), strides=(2, 2))(modelLayers)
modelLayers = layers.concatenate([branch3x3, branch3x3dbl, branch_pool], axis=channel_axis, name="mixed3")

# mixed 4: 17 modelLayers 17 modelLayers 768
branch1x1 = conv2d_bn(modelLayers, 192, 1, 1)

branch7x7 = conv2d_bn(modelLayers, 128, 1, 1)
branch7x7 = conv2d_bn(branch7x7, 128, 1, 7)
branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

branch7x7dbl = conv2d_bn(modelLayers, 128, 1, 1)
branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 1, 7)
branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

branch_pool = layers.AveragePooling2D((3, 3), strides=(1, 1), padding="same")(modelLayers)
branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
modelLayers = layers.concatenate(
    [branch1x1, branch7x7, branch7x7dbl, branch_pool],
    axis=channel_axis,
    name="mixed4",
)

# mixed 5, 6: 17 modelLayers 17 modelLayers 768
for i in range(2):
    branch1x1 = conv2d_bn(modelLayers, 192, 1, 1)

    branch7x7 = conv2d_bn(modelLayers, 160, 1, 1)
    branch7x7 = conv2d_bn(branch7x7, 160, 1, 7)
    branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

    branch7x7dbl = conv2d_bn(modelLayers, 160, 1, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 1, 7)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

    branch_pool = layers.AveragePooling2D((3, 3), strides=(1, 1), padding="same")(modelLayers)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
    modelLayers = layers.concatenate(
        [branch1x1, branch7x7, branch7x7dbl, branch_pool],
        axis=channel_axis,
        name="mixed" + str(5 + i),
    )

# mixed 7: 17 modelLayers 17 modelLayers 768
branch1x1 = conv2d_bn(modelLayers, 192, 1, 1)

branch7x7 = conv2d_bn(modelLayers, 192, 1, 1)
branch7x7 = conv2d_bn(branch7x7, 192, 1, 7)
branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

branch7x7dbl = conv2d_bn(modelLayers, 192, 1, 1)
branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 7, 1)
branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)
branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 7, 1)
branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

branch_pool = layers.AveragePooling2D((3, 3), strides=(1, 1), padding="same")(modelLayers)
branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
modelLayers = layers.concatenate(
    [branch1x1, branch7x7, branch7x7dbl, branch_pool],
    axis=channel_axis,
    name="mixed7",
)

# mixed 8: 8 modelLayers 8 modelLayers 1280
branch3x3 = conv2d_bn(modelLayers, 192, 1, 1)
branch3x3 = conv2d_bn(branch3x3, 320, 3, 3, strides=(2, 2), padding="valid")

branch7x7x3 = conv2d_bn(modelLayers, 192, 1, 1)
branch7x7x3 = conv2d_bn(branch7x7x3, 192, 1, 7)
branch7x7x3 = conv2d_bn(branch7x7x3, 192, 7, 1)
branch7x7x3 = conv2d_bn(branch7x7x3, 192, 3, 3, strides=(2, 2), padding="valid")

branch_pool = layers.MaxPooling2D((3, 3), strides=(2, 2))(modelLayers)
modelLayers = layers.concatenate([branch3x3, branch7x7x3, branch_pool], axis=channel_axis, name="mixed8")

# mixed 9: 8 modelLayers 8 modelLayers 2048
for i in range(2):
    branch1x1 = conv2d_bn(modelLayers, 320, 1, 1)

    branch3x3 = conv2d_bn(modelLayers, 384, 1, 1)
    branch3x3_1 = conv2d_bn(branch3x3, 384, 1, 3)
    branch3x3_2 = conv2d_bn(branch3x3, 384, 3, 1)
    branch3x3 = layers.concatenate(
        [branch3x3_1, branch3x3_2],
        axis=channel_axis,
        name="mixed9_" + str(i),
    )

    branch3x3dbl = conv2d_bn(modelLayers, 448, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 384, 3, 3)
    branch3x3dbl_1 = conv2d_bn(branch3x3dbl, 384, 1, 3)
    branch3x3dbl_2 = conv2d_bn(branch3x3dbl, 384, 3, 1)
    branch3x3dbl = layers.concatenate([branch3x3dbl_1, branch3x3dbl_2], axis=channel_axis)

    branch_pool = layers.AveragePooling2D((3, 3), strides=(1, 1), padding="same")(modelLayers)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
    modelLayers = layers.concatenate(
        [branch1x1, branch3x3, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name="mixed" + str(9 + i),
    )
    
modelWithoutweights = training.Model(img_input, modelLayers, name="inception_v3")
    
WEIGHTS_PATH_NO_TOP = (
    "https://storage.googleapis.com/tensorflow/keras-applications/"
    "inception_v3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
)
    
weights_path = data_utils.get_file(
                "inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5",
                WEIGHTS_PATH_NO_TOP,
                cache_subdir="models",
                file_hash="bcbd6486424b2319ff4ef7d526e38f63",)

modelWithoutweights.load_weights(weights_path)
modelWithweights.add(modelWithoutweights)

modelWithweights.add(GlobalAveragePooling2D())
modelWithweights.add(Flatten())
modelWithweights.add(Dense(4096,activation='relu'))
modelWithweights.add(Dropout(0.5))
modelWithweights.add(Dense(6,activation='softmax'))


print("done")

87924736/87910968 [==============================] - 3s 0us/step


2022-12-25 21:08:01.647544: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-25 21:08:01.813057: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-25 21:08:01.813885: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-25 21:08:01.815439: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

done


In [7]:
modelWithweights.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 6, 6, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 4096)              8392704   
_________________________________________________________________
dropout (Dropout)            (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 24582     
Total params: 30,220,070
Trainable params: 30,185,638
Non-trainable params: 34,432
_______________________________________

# 6. Model training

In [8]:
modelWithweights.compile(Adam(learning_rate=0.0001),
              loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])


checkpoint = ModelCheckpoint("Inception.h5", monitor='val_acc', verbose=1, save_best_only=True,
                             save_weights_only=False, mode='auto', period=1)
print("done")

done


In [9]:
history = modelWithweights.fit(trainingDataset, epochs=10, validation_data=validationnDataset, callbacks=[checkpoint])

Epoch 1/10


2022-12-25 21:08:25.766862: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


43/43 [==============================] - ETA: 0s - batch: 21.0000 - size: 31.3023 - loss: 0.8669 - acc: 0.6961

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '



Epoch 00001: val_acc improved from -inf to 0.48657, saving model to Inception.h5
43/43 [==============================] - 55s 787ms/step - batch: 21.0000 - size: 31.3023 - loss: 0.8670 - acc: 0.6961 - val_loss: 1.6985 - val_acc: 0.4866
Epoch 2/10
43/43 [==============================] - ETA: 0s - batch: 21.0000 - size: 31.3023 - loss: 0.3262 - acc: 0.8945
Epoch 00002: val_acc improved from 0.48657 to 0.88657, saving model to Inception.h5
43/43 [==============================] - 29s 694ms/step - batch: 21.0000 - size: 31.3023 - loss: 0.3259 - acc: 0.8945 - val_loss: 0.3902 - val_acc: 0.8866
Epoch 3/10
43/43 [==============================] - ETA: 0s - batch: 21.0000 - size: 31.3023 - loss: 0.2384 - acc: 0.9235
Epoch 00003: val_acc improved from 0.88657 to 0.94030, saving model to Inception.h5
43/43 [==============================] - 31s 731ms/step - batch: 21.0000 - size: 31.3023 - loss: 0.2382 - acc: 0.9235 - val_loss: 0.1943 - val_acc: 0.9403
Epoch 4/10
43/43 [=======================

# 7. Model Prediction

In [10]:
from keras.models import load_model
loadedModel = load_model('/kaggle/working/Inception.h5')
print("Done")

Done


In [11]:
import os
os.chdir(r'/kaggle/working/')
from IPython.display import FileLink
FileLink(r'/kaggle/working/Inception.h5')


/kaggle/working/Inception.h5

In [12]:
with open('Inception.csv', 'w', encoding='UTF8',newline='') as CsvWriter:
  header = ["image_name","label"]
  writer = csv.writer(CsvWriter)
  writer.writerow(header)
  for img in tqdm(os.listdir(test_dir)):
    path = os.path.join(test_dir, img)
    img_arr = tf.keras.utils.load_img(path,target_size=(IMG_SIZE,IMG_SIZE))
    img_arr = np.asarray(img_arr)
    img_arr = np.expand_dims(img_arr, axis=0)
    prediction = loadedModel.predict(img_arr)
    data = [img,np.argmax(prediction)]
    writer.writerow(data)

100%|██████████| 688/688 [00:24<00:00, 28.31it/s]
